In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import warnings 
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier as model
import pickle
import math
import itertools
import joblib
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier as RF
from xgboost import XGBClassifier as XGB
from numpy import hstack
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import sklearn.metrics as metrics
warnings.filterwarnings("ignore")


In [ ]:
X_TRAIN=pd.read_csv('../../New Folder/final.csv').drop('y',axis=1)
Y_TRAIN=pd.read_csv('../../New Folder/final.csv')['y']

X_VALIDATION=pd.read_csv('../../New Folder/val.csv').drop('y',axis=1)
Y_VALIDATION=pd.read_csv('../../New Folder/val.csv')['y']


X_TEST=pd.read_csv('../../New Folder/test.csv').drop('y',axis=1)
Y_TEST=pd.read_csv('../../New Folder/test.csv')['y']


In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(24,20))

# Entire DataFrame
corr = X_TRAIN.corr()
sns.heatmap(corr, cmap='coolwarm_r', annot_kws={'size':20}, ax=ax1)
ax1.set_title("Imbalanced Correlation Matrix Train)", fontsize=14)


sub_sample_corr = X_VALIDATION.corr()
sns.heatmap(sub_sample_corr, cmap='coolwarm_r', annot_kws={'size':20}, ax=ax2)
ax2.set_title(' Correlation Matrix Val', fontsize=14)
plt.show()

In [ ]:
print(X_TRAIN.shape,Y_TRAIN.shape)
print(X_VALIDATION.shape,Y_VALIDATION.shape)
print(X_TEST.shape,Y_TEST.shape)

min_max_scaler = MinMaxScaler().fit(X_TRAIN) # Fit On Training Data
X_TRAIN = min_max_scaler.transform(X_TRAIN) # Transform On Training Data
X_VALIDATION= min_max_scaler.transform(X_VALIDATION) # Transform On Validation Data
X_TEST= min_max_scaler.transform(X_TEST) # Transform On Testing Data

In [ ]:
model_1_XGBOOST = joblib.load('model_1XGB.pkl')
model_1_RF = joblib.load('model_1RF.pkl')
model_1_SVM = joblib.load('model_1SVM.pkl') 

In [ ]:
XGB_val=model_1_XGBOOST.predict(X_VALIDATION)
RF_val=model_1_RF.predict(X_VALIDATION)
SVM_val=model_1_SVM.predict(X_VALIDATION)
XGB_test=model_1_XGBOOST.predict(X_TEST)
RF_test=model_1_RF.predict(X_TEST)
SVM_test=model_1_SVM.predict(X_TEST)


In [ ]:
def predict_classes(probability_array):
    for i in range(len(probability_array)):
        if (probability_array[i]>0.5):
            probability_array[i]=1
        else:
            probability_array[i]=0
    return (probability_array) 

In [ ]:
def accuracy_check(AY_VALIDATE,AX_VALIDATE):

    abc=confusion_matrix(AY_VALIDATE,AX_VALIDATE,normalize='true')

    TN=abc[0][0]
    FP=abc[0][1]
    FN=abc[1][0]
    TP=abc[1][1]

    PS=FN+TP
    NS=TN+FP
    Accuracy           = ((TP+TN)/(PS+NS))*100
    Sensitivity        = (TP/PS)*100
    Precision          = (TP/(FP+TP))*100
    F1                 = (2*Sensitivity*Precision)/(Sensitivity+Precision )
    Speciﬁcity         = (TN/NS)*100
    MCC                = (( (TP * TN) - (FP * FN )) /  math.sqrt (( TP + FP )*( TP + FN )*( TN + FP )*( TN + FN )))
    print('Accuracy:' ,Accuracy)
    print('Sensitivity:', Sensitivity)
    print('Precision:' ,Precision)
    print('Matthews',MCC)

    cmap=plt.cm.Greens
    title='Confusion matrix'
    cm=abc
    classes = ['No-Landslide', 'Landslide']
    normalize=True

    plt.figure(figsize = (10, 10))
    plt.imshow(cm, cmap=cmap)
    plt.title(title, size = 24)
    plt.colorbar(aspect=4)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, size = 14)
    plt.yticks(tick_marks, classes, size = 14)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    # Label the plot
    for i, j in itertools.product(range(cm.shape[0]),   range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), 
             fontsize = 20,
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")
        
    # plt.grid()
    plt.tight_layout()
    plt.ylabel('True label', size = 18)
    plt.xlabel('Predicted label', size = 18)

In [ ]:
accuracy_check(Y_TEST,model_1_XGBOOST.predict(X_TEST))

In [ ]:
accuracy_check(Y_VALIDATION,model_1_XGBOOST.predict(X_VALIDATION))

In [ ]:
accuracy_check(Y_VALIDATION,model_1_RF.predict(X_VALIDATION))

In [ ]:
accuracy_check(Y_TEST,model_1_SVM.predict(X_TEST))

In [ ]:

n_estimators = 1000
max_features = 'sqrt'

max_depth=11
learning_rate=0.001
kernel ='rbf'
C = 1.0 # default=1.0


def get_models():
	models = list()
	models.append(('RF', RF(n_estimators=n_estimators,max_features='sqrt',n_jobs=16)))
	models.append(('XGB',XGB(max_depth=max_depth,learning_rate=learning_rate,n_estimators=500,n_jobs=20)))
	models.append(('svm', SVC(kernel=kernel,C=C,probability=True)))
	return models

def fit_ensemble(models, X_train, X_val, y_train, y_val):
	# fit all models on the training set and predict on hold out set
	meta_X = list()
	for name, model in models:
		# fit in training set
		model.fit(X_train, y_train)
		# predict on hold out set
		yhat = model.predict(X_val)
		# reshape predictions into a matrix with one column
		yhat = yhat.reshape(len(yhat), 1)
		# store predictions as input for blending
		meta_X.append(yhat)
	# create 2d array from predictions, each set is an input feature
	meta_X = hstack(meta_X)
	# define blending model
	blender = LogisticRegression()
	# fit on predictions from base models
	blender.fit(meta_X, y_val)
	return blender

def predict_ensemble(models, blender, X_test):
	# make predictions with base models
	meta_X = list()
	for name, model in models:
		# predict with base model
		yhat = model.predict(X_test)
		# reshape predictions into a matrix with one column
		yhat = yhat.reshape(len(yhat), 1)
		# store prediction
		meta_X.append(yhat)
	# create 2d array from predictions, each set is an input feature
	meta_X = hstack(meta_X)
	# predict
	return blender.predict_proba(meta_X)

X_train, X_val, y_train, y_val = train_test_split(X_TRAIN, Y_TRAIN, test_size=0.33, random_state=1)

models = get_models()
# train the blending ensemble
blender = fit_ensemble(models, X_train, X_val, y_train, y_val)
# make predictions on test set
yhat = predict_ensemble(models, blender, X_TEST)


In [ ]:
clas=predict_classes(yhat[:,1])

In [ ]:
accuracy_check(Y_TEST,clas)

In [ ]:
ensemble_val=predict_ensemble(models,blender,X_TEST)



In [ ]:
def roc_curve(prob,y):
    fpr, tpr, threshold = metrics.roc_curve(y,prob[:,1])
    roc_auc = metrics.auc(fpr, tpr)
    return(fpr,tpr,roc_auc)

In [ ]:
fpr1,tpr1,roc_auc1=roc_curve(model_1_RF.predict_proba(X_TEST),Y_TEST)
fpr2,tpr2,roc_auc2=roc_curve(model_1_XGBOOST.predict_proba(X_TEST),Y_TEST)
fpr3,tpr3,roc_auc3=roc_curve(model_1_SVM.predict_proba(X_TEST),Y_TEST)
fpr4,tpr4,roc_auc4=roc_curve(ensemble_val.predict_proba(X_TEST),Y_TEST)


In [ ]:

fig =plt.figure(figsize=(15,10))
ax=plt.axes()
plt.title('AUC ROC curves',size=40)
plt.plot(fpr1, tpr1, 'b', label = 'AUC SVM = %0.3f' % roc_auc1)
plt.plot(fpr2, tpr2, 'r', label = 'AUC XGBOOST = %0.3f'% roc_auc2)
plt.plot(fpr3, tpr3, 'g', label = 'AUC RF = %0.3f' % roc_auc3)
plt.plot(fpr4, tpr4, 'g', label = 'AUC RF = %0.3f' % roc_auc4)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate',size=20)
plt.xlabel('False Positive Rate',size=20)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
feature_scores = model_1_RF.feature_importances_ 
index=[ 'NDVI', 'TWI', 'Downslope', 'Upslope', 'Curvature',
       'Elevation', 'Precipitation', 'Slope', 'Clay', 'Sand', 'Silt', 'Lineament',
       'Road', 'Rivers', 'Soil_depth', 'R_factor', 'Aspect'].sort_values(ascending=False)
sns.barplot(x=feature_scores, y=feature_scores.index)
# Add labels to the graph
plt.xlabel('Feature Importance Score',size=20)
plt.ylabel('Landslide Conditioning Factors',size=18)
# Add title to the graph
# Visualize the graph
plt.savefig('./RF_imp.jpg',dpi=400)
plt.show()